# Explosion detector

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Define the coordinate system

In [ ]:
NUM_POINTS = 5000
LAPS = 50

radius = np.arange(1, NUM_POINTS + 1) / NUM_POINTS
theta = LAPS * 2 * np.pi * radius
spiral_x = radius * np.cos(theta)
spiral_y = radius * np.sin(theta)
spiral_locations = np.column_stack((spiral_x, spiral_y))

Define the locations of the detection stations on the surface:

In [ ]:
NUM_SENSORS = 5
sensors = np.arange(1, NUM_SENSORS + 1)
theta_sensor = 2 * np.pi * sensors / NUM_SENSORS
sensors_x = np.cos(theta_sensor)
sensors_y = np.sin(theta_sensor)
sensors_locations = np.column_stack((sensors_x, sensors_y))

Plot all locations:

In [ ]:
plt.figure(figsize=(12, 12))
plt.axis('equal')
plt.grid()

# Spiral locations
plt.plot(spiral_x, spiral_y, alpha=0.25)
plt.scatter(spiral_x, spiral_y, 2)

# Stations
plt.scatter(sensors_x, sensors_y, 50, label='Sensors')

plt.legend()
plt.show()

Define what value on each sensor would be generated by an explosion at internal spiral location:

In [ ]:
def detected_values(earthquake_locations, sensor_location):
    diff = earthquake_locations - sensor_location
    d = np.linalg.norm(diff, axis=1)
    measured = 1 / (d**2 + 0.1)
    return measured

In [ ]:
v = np.empty((NUM_POINTS, NUM_SENSORS))
for i, sensor_location in enumerate(sensors_locations):
    v[:, i] = detected_values(spiral_locations, sensor_location)

Make the explosion data:

In [ ]:
true_earthquake_idx = np.random.choice(len(spiral_locations))
true_earthquake_location = spiral_locations[true_earthquake_idx]

Standard deviation $\sigma$ of the Gaussian noise:

In [ ]:
sigma = 1

Get the noisy sensor values that will be observed for this explosion:

In [ ]:
values_clean = detected_values(true_earthquake_location, sensors_locations)
values_noisy = values_clean + sigma * np.random.randn(NUM_SENSORS)

In [ ]:
plt.figure()
plt.grid()

plt.plot(values_clean, 'o', label='Theoretical')
plt.plot(values_noisy, 'o', label='Observed')
plt.xticks(range(NUM_SENSORS))

plt.legend()
plt.show()

Perform inference $p(\text{location}|\text{observed sensor values})$ given these sensor values:

In [ ]:
# This is not optimised :(
logp = np.empty((NUM_POINTS, 1))
for spiral_idx in range(NUM_POINTS):
    for sensor_idx in range(NUM_SENSORS):
        # Gaussian distribution
        logp[spiral_idx] += -0.5 * (values_noisy[sensor_idx] - v[spiral_idx, sensor_idx])**2 / (sigma**2)
p = np.exp(logp - logp.max()) # do exponentiation (and avoid over/underflow)
p /= p.sum() # normalise